In [1]:
%cd Projects/loan_default_predictor/src

/teamspace/studios/this_studio/Projects/loan_default_predictor/src


In [2]:
from data.preprocessing import *
from data.cleaning import *
from data.utils import *
from modeling.model_factory import *
from modeling.utils import *
from training.training import *
from training.utils import *
from evaluation.evaluation import *
from evaluation.utils import *

In [3]:
train_data = pd.read_csv("/teamspace/studios/this_studio/Projects/loan_default_predictor/input/train.csv")
test_data = pd.read_csv("/teamspace/studios/this_studio/Projects/loan_default_predictor/input/test.csv")

In [4]:
t = [
    ("rate_of_interest","total_units"),
    ("income","total_units"),
    ("Interest_rate_spread","loan_type"),
    ("Upfront_charges","total_units"),
    ("term","total_units"),
    ("dtir1","total_units"),
    ("property_value","loan_type"),
    ("LTV","loan_type")
    ]

In [5]:
clean_train, cols_to_drop, num_imp, cat_imp = clean_training_data(train_data,0.95,t)

In [6]:
clean_test = clean_test_data(test_data,cols_to_drop,num_imp,cat_imp)

In [7]:
train_data_dict = split_features_label(data=clean_train, label_name="Status")
test_data_dict = split_features_label(data=clean_test, label_name="Status")

In [8]:
prepro_config = {"numerical":{"MinMax":["Credit_Score","term"],
"StandardScaler":["rate_of_interest","Interest_rate_spread","dtir1"],
"PowerTransformer":["loan_amount","LTV","property_value","income","Upfront_charges"]},
"categorical":{"OneHotEncoder":['loan_limit', 'Gender', 'approv_in_adv', 'loan_type', 'loan_purpose',
       'Credit_Worthiness', 'open_credit', 'business_or_commercial',
       'Neg_ammortization', 'interest_only', 'lump_sum_payment',
       'construction_type', 'occupancy_type', 'Secured_by', 'total_units',
       'credit_type', 'co-applicant_credit_type', 'age',
       'submission_of_application', 'Region', 'Security_Type']}}

In [9]:
preprocessor = build_preprocessor(prepro_config)

In [10]:
model_config = {"name":"XGB","params":{"n_estimators": 100,
            "num_leaves": 8,
            "max_leaves": 8,
            "random_state":1,
            "max_depth": 5}}

In [11]:
model = build_model(model_config)

In [12]:
pipeline, score = train(train_data_dict,preprocessor,model,5,"f1")

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:10:39] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:10:43] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:10:47] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboost/core.py:160: UserWarning: [09:10:51] WARNING: /workspace/src/learner.cc:742: 
Parameters: { "num_leaves" } are not used.

  warnings.warn(smsg, UserWarning)
/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/xgboo

In [13]:
o = evaluate(pipeline,test_data_dict)

In [14]:
o

{'accuracy': 0.9999663684670748,
 'recall': 1.0,
 'precision': 0.999862618491551,
 'f1_score': 0.9999313045270316,
 'AUC': 0.9999777342358389}